In [1]:
import requests

# Card data from the-fab-cube
url = "https://raw.githubusercontent.com/the-fab-cube/flesh-and-blood-cards/develop/json/english/card.json"

card_data = requests.get(url).json()

In [2]:
class Card:
    def __init__(self, card_info):
        # Assigning basic card properties
        self.unique_id = card_info["unique_id"]
        self.name = card_info["name"]
        # Try converting to integer, assign 0 if it fails
        converted_pitch = str(card_info["pitch"]).replace("X", "").replace("*","")
        try:
            self.pitch = int(converted_pitch) if converted_pitch != "" else 0
        except ValueError:
            self.pitch = 0

        # Similarly for other properties
        converted_cost = str(card_info["cost"]).replace("X", "").replace("*","")
        try:
            self.cost = int(converted_cost) if converted_cost != "" else 0
        except ValueError:
            self.cost = 0

        converted_power = str(card_info["power"]).replace("X", "").replace("*","")
        try:
            self.power = int(converted_power) if converted_power != "" else 0
        except ValueError:
            self.power = 0

        converted_defense = str(card_info["defense"]).replace("X", "").replace("*","")
        try:
            self.defense = int(converted_defense) if converted_defense != "" else 0
        except ValueError:
            self.defense = 0
        self.health = card_info["health"]
        self.intelligence = card_info["intelligence"]
        self.types = card_info["types"]
        self.card_keywords = card_info["card_keywords"]
        self.abilities_and_effects = card_info["abilities_and_effects"]
        self.ability_and_effect_keywords = card_info["ability_and_effect_keywords"]
        self.granted_keywords = card_info["granted_keywords"]
        self.removed_keywords = card_info["removed_keywords"]
        self.interacts_with_keywords = card_info["interacts_with_keywords"]
        self.functional_text = card_info["functional_text"]
        self.functional_text_plain = card_info["functional_text_plain"]
        self.type_text = card_info["type_text"]
        self.played_horizontally = card_info["played_horizontally"]
        self.blitz_legal = card_info["blitz_legal"]
        self.cc_legal = card_info["cc_legal"]
        self.commoner_legal = card_info["commoner_legal"]
        self.blitz_living_legend = card_info["blitz_living_legend"]
        self.cc_living_legend = card_info["cc_living_legend"]
        self.blitz_banned = card_info["blitz_banned"]
        self.cc_banned = card_info["cc_banned"]
        self.commoner_banned = card_info["commoner_banned"]
        self.upf_banned = card_info["upf_banned"]
        self.blitz_suspended = card_info["blitz_suspended"]
        self.cc_suspended = card_info["cc_suspended"]
        self.commoner_suspended = card_info["commoner_suspended"]
        self.ll_restricted = card_info["ll_restricted"]

        # Handling True/False conditions about card properties in objects.
        self.properties = CardProperties(card_info["functional_text_plain"])

        # Handling printings as a list of dictionaries
        self.printings = [Printing(p) for p in card_info["printings"]]

    def __str__(self):
        return f"{self.name} ({self.pitch})"
        
    def __repr__(self):
        return f"{self.name} ({self.pitch})"
    
class CardProperties:
    def __init__(self, functional_text_plain):
        self.has_boost = "Boost" in functional_text_plain
        self.has_go_again = "Go again" in functional_text_plain.split("\n")

    def __repr__(self):
        properties_dict = {prop: getattr(self, prop) for prop in vars(self)}
        return f"{properties_dict}"

    def __str__(self):
        properties_dict = {prop: getattr(self, prop) for prop in vars(self)}
        return f"{properties_dict}"



class Printing:
    def __init__(self, printing_info):
        self.unique_id = printing_info["unique_id"]
        self.set_printing_unique_id = printing_info["set_printing_unique_id"]
        self.id = printing_info["id"]
        self.set_id = printing_info["set_id"]
        self.edition = printing_info["edition"]
        self.foiling = printing_info["foiling"]
        self.rarity = printing_info["rarity"]
        self.artist = printing_info["artist"]
        self.art_variation = printing_info["art_variation"]
        self.flavor_text = printing_info["flavor_text"]
        self.flavor_text_plain = printing_info["flavor_text_plain"]
        self.image_url = printing_info["image_url"]



all_cards = []
for card in card_data:
    card_obj = Card(card)
    all_cards.append(card_obj)


In [3]:
import itertools
import numpy as np
from tqdm import tqdm
from collections import Counter
from collections.abc import Iterable as ABCIterable

from typing import Tuple, List, Callable, Iterable, Union, Optional, Any


class Hand:
    def __init__(self, hand_cards_tuple: tuple, arsenal_card: Card, count: int):
        self.hand = hand_cards_tuple
        self.arsenal = arsenal_card
        self.count = count
    
    def __iter__(self):
        return iter(self.hand)
    
    def has_arsenal(self) -> bool:
        return self.arsenal is not None
    
    def arsenal_is(self, cards) -> bool:
        if not isinstance(cards, ABCIterable):
            cards = [cards]
        return self.arsenal in cards
    
    def has_n_cards(self, n: int) -> bool:
        return len(self.hand) == n
    
    
    def hand_has_cards(self, cards, condition: Union[Callable[[Iterable], bool], None] = None) -> bool:
        """
        Checks if the hand contains cards based on a specified condition (e.g., any, all),
        or checks for any match if a single card is passed.

        Args:
            cards: A single card or an iterable of cards to check against the hand.
            condition (Union[Callable[[Iterable], bool], None]): A function that takes an iterable and returns a boolean
                (e.g., any, all). Defaults to None, which implies 'any' for single card inputs.

        Returns:
            bool: The result of applying the condition function to the presence checks of the specified cards.
        """
        if not isinstance(cards, ABCIterable):
            # Single card is passed, default to using 'any'
            return any(card in self.hand for card in [cards])
        
        if condition is None:
            raise ValueError("A condition function must be provided for iterable inputs.")

        return condition(card in self.hand for card in cards)


    def hand_lacks_cards(self, cards, condition: Union[Callable[[Iterable], bool], None] = None) -> bool:
        """
        Checks if the hand does not contain any of the specified cards.

        Args:
            cards: A single card or an iterable of cards to check against the hand.
            condition (Union[Callable[[Iterable], bool], None]): A function that takes an iterable and returns a boolean
                (e.g., any, all). For this method, it should be used to confirm the absence of all specified cards.

        Returns:
            bool: The result of applying the condition function to the absence checks of the specified cards.
        """
        if not isinstance(cards, ABCIterable):
            # Single card is passed
            return all(card not in self.hand for card in [cards])
        
        if condition is None:
            # If no condition is provided, check that none of the cards are in the hand
            return all(card not in self.hand for card in cards)

        # Apply the provided condition to check for the absence of cards
        return condition(card not in self.hand for card in cards)
    
    def __eq__(self, other):
        """Check if two Hand objects are equal based on their attributes."""
        if not isinstance(other, Hand):
            return False
        return sorted(self.hand, key=lambda card: card.unique_id) == sorted(other.hand, key=lambda card: card.unique_id) and self.arsenal == other.arsenal

    def __hash__(self):
        """Return the hash value of a Hand object."""
        return hash((self.hand, self.arsenal, self.count))

    def __getitem__(self, key):
        return self.hand[key]
    
    def __len__(self) -> int:
        return len(self.hand)
    
    def __str__(self) -> str:
        return f"[{self.count}] Hand: {self.hand}; Arsenal: {self.arsenal}"
    
    def __repr__(self) -> str:
        return f"[{self.count}] Hand: {self.hand}; Arsenal: {self.arsenal}"


class HandSet:
    def __init__(self, hand_list):
        self.hands = hand_list

    def difference(self, other_handset: 'HandSet') -> 'HandSet':
        """
        Compares this HandSet to another HandSet and returns a new HandSet containing
        hands that are in this HandSet but not in the other.

        Args:
            other_handset (HandSet): Another HandSet object to compare with.

        Returns:
            HandSet: A new HandSet containing hands unique to this HandSet.
        """
        set_self = set(self.hands)
        set_other = set(other_handset.hands)
        unique_hands = set_self - set_other
        return HandSet(list(unique_hands))
    

    def compare_size_to(self, other_handset: 'HandSet') -> float:
            """
            Calculates the percentage of this HandSet's total hand count relative to another HandSet.

            Args:
                other_handset (HandSet): Another HandSet object to compare with.

            Returns:
                float: The percentage of this HandSet's total count relative to the other HandSet.
            """
            total_count_self = sum(hand.count for hand in self.hands)
            total_count_other = sum(hand.count for hand in other_handset.hands)

            if total_count_other == 0:
                return 0.0  # Avoid division by zero

            return (total_count_self / total_count_other)
    
    def filter(self, condition_method_name: str, *args, **kwargs) -> 'HandSet':
        """
        Filters hands based on a provided condition function.

        Args:
            condition_function (Callable[[Hand], bool]): A function that takes a Hand object
                and returns True if the condition is met, False otherwise.

        Returns:
            HandSet: A new HandSet object containing hands that meet the condition.
        """
        filtered_hands = [hand for hand in self.hands if getattr(hand, condition_method_name)(*args, **kwargs)]
        return HandSet(filtered_hands)
    
    def __iter__(self):
        return iter(self.hands)

    def __getitem__(self, key):
        return self.hands[key]
    
    def __str__(self) -> str:
        return f"Handset object with {len(self.hands)} distinct hands, and {sum([hand.count for hand in self.hands])} hand combinations."
    
    def __repr__(self) -> str:
        return f"Handset object with {len(self.hands)} distinct hands, and {sum([hand.count for hand in self.hands])} hand combinations."
    
    def __len__(self) -> int:
        return len(self.hands)
    

class Deck:
    def __init__(self, deck_string, card_database):
        self.cards = []
        self.parse_deck_string(deck_string, card_database)

    def group_by(self, condition):
        """
        Groups cards based on the provided condition.
        condition: a lambda function that returns a boolean.
        """
        grouped_cards = [card for card in self.cards if condition(card)]
        return grouped_cards

    def parse_deck_string(self, deck_string, card_database):
        lines = deck_string.split('\n')
        for line in lines:
            if '(' in line and ')' in line:
                quantity, rest = line.split(')', 1)
                quantity = int(quantity.strip('('))
                name, pitch_color = rest.rsplit(' ', 1)
                name = name.strip()
                pitch = self.color_to_pitch(pitch_color)

                card = self.find_card_in_database(card_database, name, pitch)
                if card:
                    for _ in range(quantity):
                        self.cards.append(card)

    @staticmethod
    def color_to_pitch(color):
        return {'(red)': 1, '(yellow)': 2, '(blue)': 3}.get(color, '')

    @staticmethod
    def find_card_in_database(card_database, name, pitch):
        # Assuming card_database is a list of Card objects
        for card in card_database:
            if card.name == name and card.pitch == pitch:
                return card
        return None
    

    def calculate_hands(self):
        hand_counts = Counter()
        deck_counts = Counter(self.cards)

        # Generate combinations for hand sizes 1, 2, 3, and 4
        for hand_size in range(0, 5):
            print(f"Calculating permutations for hand size {hand_size}")
            total_combinations = itertools.combinations(self.cards, hand_size)

            for combination in tqdm(total_combinations):
                # Count occurrences of each card in the combination
                combination_counts = Counter(combination)
                sorted_combination = tuple(sorted(combination, key=lambda card: card.unique_id))

                # Count the hand without any arsenal card
                hand_counts[(sorted_combination, None)] += 1

                # Append each possible arsenal card to the sorted combination
                for arsenal_card in self.cards:
                    if deck_counts[arsenal_card] > combination_counts[arsenal_card]:
                        full_hand = (sorted_combination, arsenal_card)
                        hand_counts[full_hand] += 1

        # Create a list to store the hands and their counts
        hands_list = []

        # Fill the list with hands and their counts
        for hand, count in hand_counts.items():
            hand_obj = Hand(hand[0], hand[1], count)
            hands_list.append((hand_obj))  # Appending the count to the hand tuple

        
        return HandSet(hands_list)

    def __str__(self):
        return f"Deck with {len(self.cards)} cards"
    
    def __repr__(self):
        return f"Deck with {len(self.cards)} cards"


In [4]:
from typing import List, Tuple, Optional
from itertools import product
from collections import Counter

def generate_handsets(deck: Deck, card_options_tuple: Tuple[List[Card], ...], arsenal_options: Optional[List[Card]] = None) -> Tuple[HandSet, int]:
    """
    Optimized generation of handsets from a deck, considering sorted hand combinations and arsenal options.

    Args:
        deck (Deck): The deck to generate hands from.
        card_options_tuple (Tuple[List[Card], ...]): Options for each card slot in the hand.
        arsenal_options (Optional[List[Card]]): Options for the arsenal card.

    Returns:
        Tuple[HandSet, int]: Generated handsets and total number of combinations.
    """

    deck_counter = Counter(deck.cards)
    hand_counts = Counter()

    # Generate all possible hand combinations, including overcounted ones
    for raw_hand_combination in product(*card_options_tuple):
        sorted_hand_combination = tuple(sorted(raw_hand_combination, key=lambda card: card.unique_id))
        
        # Consider each arsenal card option
        if arsenal_options is not None:
            for arsenal_card in arsenal_options:
                if deck_counter[arsenal_card] > 0:  # Only consider available arsenal cards
                    hand_key = (sorted_hand_combination, arsenal_card)
                    hand_counts[hand_key] += 1
        else:
            # No arsenal card
            hand_key = (sorted_hand_combination, None)
            hand_counts[hand_key] += 1

    # Prune invalid hands that exceed the available card counts
    valid_hand_counts = Counter()
    for hand, count in hand_counts.items():
        combination_counter = Counter(hand[0])
        if hand[1] is not None:  # Consider the arsenal card
            combination_counter[hand[1]] += 1

        if all(deck_counter[card] >= count for card, count in combination_counter.items()):
            valid_hand_counts[hand] = count

    # Calculate total permutations for each valid hand
    total_combinations = 0
    for hand, count in valid_hand_counts.items():
        hand_permutations = calculate_hand_permutations(deck_counter, hand[0], hand[1])
        total_combinations += hand_permutations * count

    # Convert to HandSet
    hands_list = [Hand(hand[0], hand[1], valid_hand_counts[hand]) for hand in valid_hand_counts]
    handset = HandSet(hands_list)

    return handset, total_combinations

def calculate_hand_permutations(deck_counter: Counter, hand_cards: Tuple[Card, ...], arsenal_card: Optional[Card]) -> int:
    """
    Calculate the number of distinct permutations of a given hand from the deck.

    Args:
        deck_counter (Counter): A counter of cards in the deck.
        hand_cards (Tuple[Card, ...]): The cards in the hand.
        arsenal_card (Optional[Card]): The arsenal card, if any.

    Returns:
        int: The number of distinct permutations of the hand from the deck.
    """
    hand_counter = Counter(hand_cards)
    if arsenal_card:
        hand_counter[arsenal_card] += 1

    permutations = 1
    for card, count in hand_counter.items():
        deck_count = deck_counter[card]
        for i in range(count):
            permutations *= deck_count - i

    return permutations

# Example usage
# deck = ...  # Initialize your deck
# card_options_tuple = ...  # Define your card options tuple
# arsenal_options = ...  # Optionally define your arsenal options
# handset, total_combinations = generate_handsets(deck, card_options_tuple, arsenal_options)

In [8]:
deck_string = """
Made with ❤️ at the FaBrary

e se n tiver twl

Class: Brute
Hero: Kayo, Armed and Dangerous
Weapons: Mandible Claw
Equipment: Apex Bonebreaker, Gambler's Gloves, Nullrune Gloves, Savage Sash, Scabskin Leathers, Scowling Flesh Bag, Skullhorn

(3) Bare Fangs (red)
(3) Clash of Might (red)
(3) Command and Conquer (red)
(2) Enlightened Strike (red)
(3) Pulping (red)
(3) Swing Big (red)
(3) Wild Ride (red)
(3) Bare Fangs (yellow)
(3) Beast Within (yellow)
(3) Bloodrush Bellow (yellow)
(2) Clash of Might (yellow)
(3) Send Packing (yellow)
(1) Wild Ride (yellow)
(3) Agile Windup (blue)
(3) Assault and Battery (blue)
(2) Mighty Windup (blue)
(3) Pack Call (blue)
(2) Reckless Swing (blue)
(3) Riled Up (blue)
(3) Run Roughshod (blue)
(3) Smash Instinct (blue)
(3) Wrecker Romp (blue)


See the full deck @ https://fabrary.net/decks/01J0SXWSTRDEV47SVX0KX18V71
"""

In [9]:
deck = Deck(deck_string, all_cards)
deck

Deck with 60 cards

In [10]:
bravo_full_handset = deck.calculate_hands()
bravo_full_handset = bravo_full_handset.filter('has_n_cards', 4)

Calculating permutations for hand size 0


1it [00:00, ?it/s]


Calculating permutations for hand size 1


60it [00:00, ?it/s]


Calculating permutations for hand size 2


1770it [00:00, 40301.17it/s]


Calculating permutations for hand size 3


34220it [00:00, 36537.28it/s]


Calculating permutations for hand size 4


487635it [00:13, 37282.71it/s]


In [12]:
crip = [card for card in all_cards if card.name == "Crippling Crush"][0]
rouse = [card for card in all_cards if card.name == "Rouse the Ancients"][0]


print(f"Kayo total handset: {bravo_full_handset}")
bravo_full_handset.filter('has_n_cards', 4)

Kayo total handset: Handset object with 279672 distinct hands, and 29697095 hand combinations.


Handset object with 279672 distinct hands, and 29697095 hand combinations.

In [ ]:
has_crip_AND_rouse = bravo_full_handset.filter("hand_has_cards", [crip, rouse], all)
print(has_crip_AND_rouse.compare_size_to(bravo_full_handset))
has_crip_OR_rouse = bravo_full_handset.filter("hand_has_cards", [crip, rouse], any)
print(has_crip_OR_rouse.compare_size_to(bravo_full_handset))
has_crip_OR_rouse.compare_size_to(has_crip_AND_rouse)

0.02846207684623698
0.35162068882495073


12.354006727075474

In [ ]:
bravo_full_handset.filter("hand_has_cards", crip, all).compare_size_to(bravo_full_handset)

0.19004138283559385

In [ ]:
blue_poppers_or_estrike = deck.group_by(lambda card: (card.power >= 6 and card.pitch == 3)
                                                  or (card.name == "Enlightened Strike")
                                        )


has_blue_popper_or_estrike = bravo_full_handset.filter('has_n_cards', 4).filter('hand_has_cards', blue_poppers_or_estrike, any)
has_blue_popper_or_estrike.compare_size_to(bravo_full_handset)

0.8794063527089098

In [ ]:
blue_poppers = deck.group_by(lambda card: (card.power >= 6 and card.pitch == 3)
                                          )

has_no_blue_poppers = bravo_full_handset.filter('has_n_cards', 4).filter('hand_lacks_cards', blue_poppers, all)
has_no_blue_poppers.compare_size_to(bravo_full_handset)

0.16844970863311715

In [ ]:
no_blue_popper_no_estrike = has_blue_popper_or_estrike.difference(has_no_blue_poppers)
no_blue_popper_no_estrike.compare_size_to(bravo_full_handset)

0.8315502913668829

In [ ]:
no_blue_popper_no_estrike[534]

[162] Hand: (Macho Grande (3), Sink Below (1), Unmovable (3), Imposing Visage (3)); Arsenal: Debilitate (3)

# Fai

In [6]:
fai_deck_string = """
Deck built with ❤️ at the FaBrary

World Championship: Barcelona 1st 🇬🇷 Copy

Class: Ninja
Hero: Fai, Rising Rebellion
Weapons: Harmonized Kodachi,Harmonized Kodachi, Searing Emberblade
Equipment: Flamescale Furnace, Fyendal's Spring Tunic, Mask of Momentum, Mask of the Pouncing Lynx, Snapdragon Scalers, Tide Flippers, Tiger Stripe Shuko

(3) Ancestral Empowerment (red)
(3) Bittering Thorns (red)
(3) Blaze Headlong (red)
(3) Brand with Cinderclaw (red)
(3) Double Strike (red)
(3) Enlightened Strike (red)
(3) Lava Burst (red)
(3) Mounting Anger (red)
(3) Ravenous Rabble (red)
(3) Rising Resentment (red)
(3) Ronin Renegade (red)
(3) Snatch (red)
(3) Art of War (yellow)
(3) Brand with Cinderclaw (yellow)
(3) Salt the Wound (yellow)
(3) Brand with Cinderclaw (blue)
(3) Lava Vein Loyalty (blue)
(3) Soulbead Strike (blue)
(3) Stab Wound (blue)
(2) Warmonger's Diplomacy (blue)


See the full deck @ https://fabrary.net/decks/01HH37B8K3ZDXBGCT4R7YDGDFG
"""

In [7]:
fai_deck = Deck(fai_deck_string, all_cards)
fai_full_handset = fai_deck.calculate_hands()

Calculating permutations for hand size 0


1it [00:00, ?it/s]


Calculating permutations for hand size 1


59it [00:00, 29608.03it/s]


Calculating permutations for hand size 2


...

1711it [00:00, 21657.84it/s]


Calculating permutations for hand size 3


32509it [00:01, 18232.77it/s]


Calculating permutations for hand size 4


455126it [00:24, 18585.78it/s]


In [ ]:
# We will check for 4-card hands that can cast a Lava Burst with Rupture with only 3 cards and can block 3 or more with the 4-th card. For this, we will:
# 1. Find all 4-card hands that have at least one blue
# 2. "Fill in" the other card being exactly a 0-cost draconic attack with natural Go Again (We will exclude blue draconic chain starters, but we could include those if wanted)
# 3. Filter for those hands in which the third card is a Lava Burst.
# 4. Filter for which
# Finally, we can see which percentage of 3-card hands the resulting HandSet is in comparison to all 3-card hands

has_3_cards = fai_full_handset.filter('has_n_cards', 3)     # Filters for hands in the complete handset that have exactly 3 cards

# Lets filter all 3 card hands that have, at least, one blue
blue_card = fai_deck.group_by(lambda card: card.pitch == 3)
has_a_blue = has_3_cards.filter('hand_has_cards', blue_card, any)

# Now, off the resulting set, lets check which handss also have a 0-cost draconic chain starter that is not a blue
non_blue_draconic_chain_starter =  fai_deck.group_by(lambda card: ("Attack" in card.types
                                                          and "Draconic" in card.types
                                                          and card.cost == 0
                                                          and "Go again" in card.functional_text_plain
                                                          and card.pitch != 3
                                                          )
                                            )
has_draconic_chain_starter = has_a_blue.filter('hand_has_cards', non_blue_draconic_chain_starter, any)

# Finally, lets check which of those have a Lava Burst in it
lava_burst = fai_deck.group_by(lambda card: card.name == "Lava Burst")
is_wanted_hand = has_draconic_chain_starter.filter('hand_has_cards', lava_burst, any)

# And now lets compare the size of that set of hands to the original 3-card hands set and print a few sample hands.
print(f"The probability of a 3-card hand being able to Rupture a Lava Burst is: {is_wanted_hand.compare_size_to(has_3_cards)}")

for hand in is_wanted_hand:
    print(hand)

The probability of a 3-card hand being able to Rupture a Lava Burst is: 0.01550475833034672
[27] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: None
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Ancestral Empowerment (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Bittering Thorns (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Blaze Headlong (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Brand with Cinderclaw (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Double Strike (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Enlightened Strike (1)
[81] Hand: (Lava Burst (1), Brand with Cinderclaw (3), Brand with Cinderclaw (1)); Arsenal: Lava Burst (1)
[81] Hand: (Lava Bu

In [ ]:
blue_poppers_or_estrike = deck.group_by(lambda card: (card.power >= 6 and card.pitch == 3)
                                                  or (card.name == "Enlightened Strike")
                                        )